# Basic Tools for Alhazen

> Simple tools to demonstrate utility and 'agentic' functionality. 

## Available Tools:

### Building, Describing, and Deleting Collections:
1. **AddCollectionFromEPMCTool**: Execute a query against the European PMC, creating a collection, downloading available full-text papers and saving expressions, items, and fragments to the database to denote that publication.
2. **DescribeCollectionCompositionTool**: Describe the composition of a collection in the database.
3. **DeleteCollectionTool**: Delete a collection from the database.

### Searching for papers in collections:
3. **List__Paged_Expressions_CollectionCTool**: Delete a collection from the database.

## Tools under development:
1. Develop queries across external data sources
2. Extract information from a collection using an LLM-based analysis to create Notes 
3. Filter a collection by an LLM-based analysis by tagging fragments with Notes
5. Report on the state of the database in terms of numbers of collections, expressions, items, and fragments
6. Prepare a report over a core research question by collecting a number of notes and synthesizing them into a report

In [ ]:
#| default_exp tools.basic

In [1]:
#| export

import os

from alhazen.core import OllamaRunner, PromptTemplateRegistry, get_langchain_llm, get_cached_gguf, \
    get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE
from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery
from alhazen.utils.langchain_utils import suppress_stdout_stderr
from alhazen.utils.output_parsers import JsonEnclosedByTextOutputParser
from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import download_full_text_paper_for_doi
from alhazen.utils.queryTranslator import QueryTranslator, QueryType
from alhazen.utils.web_robot import *
from alhazen.schema_sqla import ScientificKnowledgeCollection, \
    ScientificKnowledgeExpression, ScientificKnowledgeCollectionHasMembers, \
    ScientificKnowledgeItem, ScientificKnowledgeExpressionHasRepresentation, \
    ScientificKnowledgeFragment, ScientificKnowledgeItemHasPart, \
    InformationResource, Note, NoteIsAbout
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb
from alhazen.utils.jats_text_extractor import NxmlDoc

from langchain.tools import StructuredTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.llms import Ollama
from langchain.utilities import SerpAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field

import pandas as pd

from sqlalchemy import create_engine, exists, func
from sqlalchemy.orm import sessionmaker, aliased

from typing import Optional, Type

In [2]:
#| export
# NOTE - Use LangChain's SQL_DATABASE TOOL AS A MODEL 
# https://github.com/langchain-ai/langchain/blob/535db72607c4ae308566ede4af65295967bb33a8/libs/community/langchain_community/tools/sql_database/tool.py

class AlhazenToolMixin(BaseModel):
    '''Base tool for interacting with an Alhazen CEIFNS (pron. 'SAI-FiNS') database 
    (CEIFNS = Collection-Expression-Item-Fragment-Note-Summary).'''

    db: Ceifns_LiteratureDb = Field(exclude=True)
    ollr : Optional[OllamaRunner] = Field(exclude=True)
    llm : Optional[Ollama] = Field(exclude=True)
    llm_model : str = Field(default='mixtral')

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

In [3]:
#| export

class AddCollectionFromEPMCToolSchema(BaseModel):
    id: str = Field(description="A code that serves as the id of the collection we will add papers to.")
    name: str = Field(description="A human-readable name of the collection we will add papers to.")
    query: str = Field(description="A search query (using AND/OR/NOT statements) to be executed on the European PMC online literature repository for scientific publications.")
    full_text: bool = Field(description="A boolean parameter which is set to 'True' for the tool to search available online sources for each " + \
                            "individual paper and set to 'False' to only search for the citation record. This significantly adds to execution time.")

class AddCollectionFromEPMCTool(AlhazenToolMixin, BaseTool): 
    name = "add_collection_from_epmc_query"
    description = "This tool searches the European PMC literature database for references to biomedical scientific papers."
    args_schema: Type[AddCollectionFromEPMCToolSchema] = AddCollectionFromEPMCToolSchema

    def _run(
        self,
        id: str, 
        name: str,
        query: str,
        full_text: bool,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        """Use the tool."""
        if os.environ.get('LOCAL_FILE_PATH') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LOCAL_FILE_PATH']
        if loc[-1:] != '/':
            loc += '/'
        if os.environ.get('ALHAZEN_DB_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['ALHAZEN_DB_NAME']
        n_papers_added = 0

        try: 
            
            self.db.add_corpus_from_epmc_query(id, name, query, commit_this=False)

            skc = aliased(ScientificKnowledgeCollection)
            skc_hm = aliased(ScientificKnowledgeCollectionHasMembers)
            ske = aliased(ScientificKnowledgeExpression)
            ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
            ski = aliased(ScientificKnowledgeItem)
            ski_hp = aliased(ScientificKnowledgeItemHasPart)
            skf = aliased(ScientificKnowledgeFragment)

            q1 = self.db.session.query(skc) \
                    .filter(skc.id == str(id)) 
            c = q1.first()

            q2 = self.db.session.query(ske) \
                    .filter(skc.id == skc_hm.ScientificKnowledgeCollection_id) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc.id == str(id)) 
            for e in q2.all():
                n_papers_added += 1
                doi = e.id.replace('doi:', '')
                if full_text:
                    path = loc+db_name+'/ft/'
                    ft_exist = download_full_text_paper_for_doi(doi, path)
                    if ft_exist:
                        self.db.add_full_text_for_expression(e)
            
            expression_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) 
            expression_count = expression_q.first()[0]

            jats_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'JATSFullText')
            jats_ft_count = jats_ft_q.first()[0]
            
            pdf_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'PDFFullText')
            pdf_ft_count = pdf_ft_q.first()[0]

            n = Note(id='skc:%s.counts'%(id),
                    content='{"ske_count": %s, "jats_ft_count": %s, "pdf_ft_count": %s}' \
                        %(expression_count, jats_ft_count, pdf_ft_count),
                    format='json',
                    type='NoteAboutCollection')
            c.has_notes.append(n)        
            self.db.session.add(n)

            self.db.session.commit()
            
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'action': 'Final Answer', 
                'action_input': 'Successfully constructed a collection called `{}` containing {} papers by querying the European PMC Database.'.format(name, n_papers_added)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class DescribeCollectionCompositionToolSchema(BaseModel):
    id: str = Field(description="should be the id of the collection we wish to describe")
    
class DescribeCollectionCompositionTool(AlhazenToolMixin, BaseTool): 
    name = "add_collection_from_epmc_query"
    description = "This tool describes the contents of a Collection, in terms of counts of papers"
    args_schema: Type[DescribeCollectionCompositionToolSchema] = DescribeCollectionCompositionToolSchema

    def _run(
        self,
        id: str, 
        name: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        if os.environ.get('LOCAL_FILE_PATH') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LOCAL_FILE_PATH']
        if loc[-1:] != '/':
            loc += '/'
        if os.environ.get('ALHAZEN_DB_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['ALHAZEN_DB_NAME']

        try:      
            skc = aliased(ScientificKnowledgeCollection)
            skc_hm = aliased(ScientificKnowledgeCollectionHasMembers)
            ske = aliased(ScientificKnowledgeExpression)
            ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
            ski = aliased(ScientificKnowledgeItem)
            ski_hp = aliased(ScientificKnowledgeItemHasPart)
            skf = aliased(ScientificKnowledgeFragment)
            
            expression_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) 
            expression_count = expression_q.first()[0]

            jats_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'JATSFullText')
            jats_ft_count = jats_ft_q.first()[0]

            pdf_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'PDFFullText')
            pdf_ft_count = pdf_ft_q.first()[0]

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'action': 'Final Answer', 
                'action_input': 'The collection called `{}` contains {} papers; including {} PDF and {} JATS full text papers.'.format(name, expression_count, pdf_ft_count, jats_ft_count)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class DeleteCollectionToolSchema(BaseModel):
    id: str = Field(description="should be the id of the collection we will add papers to")

class DeleteCollectionTool(AlhazenToolMixin, BaseTool): 
    name = "delete_collection"
    description = "This deletes a collection from the database based on an id value."
    args_schema: Type[DeleteCollectionToolSchema] = DeleteCollectionToolSchema

    def _run(
        self,
        id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        """Use the tool."""
        if os.environ.get('LOCAL_FILE_PATH') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LOCAL_FILE_PATH']
        if os.environ.get('ALHAZEN_DB_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['ALHAZEN_DB_NAME']

        try: 
            
            self.db.delete_collection(id, commit_this=True)

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'action': 'Final Answer', 
                'action_input': 'Successfully deleted a collection with id:`{}`.'.format(id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("delete_collection does not support async")

In [ ]:
#| export

class RetrieveFullTextToolSchema(BaseModel):
    paper_id: str = Field(description="the digitial objecty identifier (doi) of the paper being retrieved from external sources")

class RetrieveFullTextTool(AlhazenToolMixin, BaseTool): 
    name = "retrieve_full_text_for_paper_id"
    description = "This retrieves a full text paper based on its doi, copies it to local disk, and adds it the database."
    args_schema: Type[RetrieveFullTextToolSchema] = RetrieveFullTextToolSchema

    def _run(
        self,
        paper_id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        """Use the tool."""
        if os.environ.get('LOCAL_FILE_PATH') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LOCAL_FILE_PATH']
        if os.environ.get('ALHAZEN_DB_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['ALHAZEN_DB_NAME']

        try: 

            ske = aliased(ScientificKnowledgeExpression)
            e = self.db.session.query(ske) \
                    .filter(ske.id == str(paper_id)).first()
            doi = e.id.replace('doi:', '')
            path = loc+db_name+'/ft/'
            ft_exist = download_full_text_paper_for_doi(doi, path)
            if ft_exist:
                self.db.add_full_text_for_expression(e)
                self.db.session.commit()

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'action': 'Final Answer', 
                'action_input': 'Successfully added full text for a paper with doi:`{}`.'.format(paper_id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("delete_collection does not support async")

In [ ]:
#os.environ['LOCAL_FILE_PATH'] = '/Users/gburns/alhazen/'
#os.environ['ALHAZEN_DB_NAME'] = 'temp'
#
#t = EMPCSearchTool()
#t._run(name='Stuff', 
#       query='Cryoelectron Tomography | Cryo Electron Tomography | Cryo-Electron Tomography | Cryo-ET | CryoE')

In [ ]:
#db_name = os.environ['DB_NAME']
#loc = os.environ['LLMS_TEMP_DIR']
#
#db = Ceifns_LiteratureDb(loc, db_name)
#if db.session is None:
#    session_class = sessionmaker(bind=db.engine)
#    db.session = session_class()
#
#r2 = db.session.query(func.count(ScientificKnowledgeExpression.id)) \
#                    .filter(ScientificKnowledgeCollection.id == ScientificKnowledgeCollectionHasMembers.ScientificKnowledgeCollection_id) \
#                    .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id) \
#                    .filter(ScientificKnowledgeCollection.id == str(0.0))
#print(str(r2.first()[0]))



1853
